In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import pickle
import ast
from scipy.special import gamma, factorial
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Function to calculate the 95% confidence interval
def mean_confidence_interval_95(data):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = 1.96*se
    return m, m-h, m+h, h

In [3]:
# Function to generate a random variable following a multinomial distribution
def RV_Multinomial(n, p, U):
    X = np.zeros(len(p), dtype = int)
    for i in range(n):
        for j in range(len(p)):
            if U[i] <= sum(p[k] for k in range(j+1)):
                X[j] = X[j] + 1
                break
    return(X)

In [4]:
# Class definition for a non-perishable inventory Markov Decision Process
class NonPerishableInventoryMDP(object):
    def startState(self):
        # Initial state (e.g., zero inventory)
        return 0

    def orders(self, state):
        # Returns list of valid actions (possible order sizes)
        TopUp = list(range(21))
        result = [max(x - state, 0) for x in TopUp]
        return list(set(result))

    def succProbCost(self, state, order, demand, size, prob, day, cs):
        # Compute the successor states, probabilities, and associated costs
        result = []
        for i in range(len(demand)):
            # Update state after demand
            newState = max(order - max(demand[i] - state, 0), 0)
            # Cost calculation:
            # cs[0]: fixed ordering cost
            # cs[1]: holding cost
            # cs[2]: shortage cost
            cost = cs[0] * int(order > 0) + cs[1] * max(order + state - demand[i], 0) + cs[2] * max(demand[i] - order - state, 0)
            # Compute demand probabilities
            if (demand[i] == 20):
                dprob = 1 - sum(gamma(i + size[day]) * (prob[day] ** size[day]) * (1 - prob[day]) ** i / (gamma(size[day]) * factorial(i)) for i in range(20))
                result.append((newState, dprob, cost))
            else:
                dprob = gamma(demand[i] + size[day]) * (prob[day] ** size[day]) * (1 - prob[day]) ** demand[i] / (gamma(size[day]) * factorial(demand[i]))
                result.append((newState, dprob, cost))
        return result

    def discount(self):
        # Discount factor for future costs
        return 0.95

    def states(self):
        # List of all possible states (inventory levels)
        return [i for i in range(21)]

In [5]:
# Function to perform value iteration for the Non-Perishable Inventory MDP
def NonPerishableValueIteration(mdp, demand, size, prob, cs):
    # Initialize value function for each state and day
    V = [{} for _ in range(7)]  # 7 days (one for each day of the week)
    for i in range(7):
        for state in mdp.states():
            V[i][str(state)] = 0.

    def Q(state, order, demand, day):
        # Compute the expected cost for a given state, action, and day
        return sum(dprob * (cost + mdp.discount() * V[(day + 1) % 7][str(newState)])
                   for newState, dprob, cost in mdp.succProbCost(state, order, demand, size, prob, day, cs))

    day = 0
    # Initialize new values to store updated state values during iteration
    newV = [{} for _ in range(7)]
    for i in range(7):
        for state in mdp.states():
            newV[i][str(state)] = 0.

    while True:

        for state in mdp.states():
            # Update value for each state by minimizing over all possible actions
            newV[day][str(state)] = min(Q(state, order, demand, day) for order in mdp.orders(state))

        # Check for convergence
        if (day == 6):
            if max(abs(V[day][str(state)] - newV[day][str(state)]) for day in range(7) for state in mdp.states()) < 0.1:
                break
            else:
                V = newV
                day = 0
                # Reinitialize newV for the next iteration
                newV = [{} for _ in range(7)]
                for i in range(7):
                    for state in mdp.states():
                        newV[i][str(state)] = 0.
        else:
            day += 1

    return V

In [6]:
# Function to calculate the approximate cost
def ExpDisCost(state, order, day, b, rep=100, alpha=0.95):
    x_list = []
    rng1 = np.random.RandomState(seed=1)
    rng2 = np.random.RandomState(seed=2)
    # Use CRN in each replication
    U = rng2.random((rep, 20))

    for i in range(rep):
        # Generate demand
        demand = rng1.negative_binomial(size[day], prob[day])
        if demand > 20:
            demand = 20

        P = []
        for i in range(m):
            if i < m - 1:
                P.append(np.exp(c0[i] + c1[i] * order) / (1 + sum(np.exp(c0[j] + c1[j] * order) for j in range(m - 1))))
            else:
                P.insert(0, 1 - sum(P))
        y = RV_Multinomial(order, P, U[i])

        # Update the inventory state
        newState = [0 for i in range(m - 1)]
        for j in range(m - 2):
            newState[m - 2 - j] = max(
                state[m - 3 - j] + y[j + 1] - max(demand - sum(state[m - 2 - k] + y[k] for k in range(j + 1)), 0), 0)
        newState[0] = max(y[m - 1] - max(demand - sum(state[m - 2 - k] + y[k] for k in range(m - 1)), 0), 0)

        if str(newState) not in Vhat[(day + 1) % 7]:
            data = [1]
            data.append(VF.iloc[(day+1)%7][sum(newState)])
            data.append(newState[0])
            data.append(newState[1])
            data.append(newState[2])
            data.append(newState[3])
            data.append(newState[0] ** 2)
            data.append(newState[1] ** 2)
            data.append(newState[2] ** 2)
            data.append(newState[3] ** 2)
            Vhat[(day + 1) % 7][str(newState)] = np.matmul(np.array(data), np.array(b[(day + 1) % 7]))

        # Calculate the one period cost plus the discounted future cost
        cost = cs[0] * int(order > 0) + cs[1] * max(order + sum(state) - demand, 0) + cs[2] * max(demand - order - sum(state), 0) + cs[3] * max(state[m - 2] + y[0] - demand, 0) + alpha * Vhat[(day + 1) % 7][str(newState)]
        x_list.append(cost)

    return np.mean(x_list)

In [7]:
# Function to calculate the approximate order
def ApproximateDecision(dow, s, b):
    TopUp = list(range(21))
    y = list(set([max(x - sum(s), 0) for x in TopUp]))
    return min((ExpDisCost(s, order, dow, b), order) for order in y)[1]

## Endog./Exog. Policy

In [8]:
# read out the optimal policy
open_file = open("ADPerf_ExcMyp_m5_CS.pkl", "rb")
ls = pickle.load(open_file)
open_file.close()

In [9]:
ls[15]['CS']

['[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]',
 '[10, 1, 20, 2]']

In [10]:
ls[15]['Endog.']

['[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]',
 '[1.6, 2.6, 2.8, 1.6, 0.0, 0.0, 0.0, 0.0]']

In [11]:
for i in range(11):
    print(mean_confidence_interval_95(ls[15]['Cost'][i]))

(319.3137618143541, 316.07100671952577, 322.55651690918245, 3.242755094828332)
(312.7415824951767, 309.91569759861636, 315.56746739173707, 2.8258848965603605)
(311.0619926091429, 308.2664049956615, 313.85758022262434, 2.7955876134814193)
(310.6243120937067, 307.82523154695707, 313.4233926404563, 2.799080546749615)
(310.2439547547096, 307.44986494941617, 313.038044560003, 2.7940898052934005)
(309.71734794869764, 306.9635469367159, 312.4711489606794, 2.7538010119817775)
(309.7317772957756, 306.9757300790987, 312.4878245124525, 2.7560472166768877)
(309.57799613457047, 306.8151199047349, 312.34087236440604, 2.7628762298355762)
(309.714873396404, 306.94662342601777, 312.48312336679027, 2.768249970386258)
(309.2115797732454, 306.44440609655624, 311.9787534499345, 2.767173676689148)
(308.8562103087523, 306.0884905372743, 311.62393008023025, 2.7677197714779735)


In [12]:
b = ls[15]['b'][10]
pi = ls[15]['pi'][10]
Vhat = [{} for i in range(7)]

In [13]:
Plogit = ast.literal_eval(ls[15]['Endog.'][10])
c0 = [Plogit[0], Plogit[1], Plogit[2], Plogit[3]]
c1 = [Plogit[4], Plogit[5], Plogit[6], Plogit[7]]

In [14]:
cs = [10, 1, 20, 2]
size = [3.497361, 10.985837, 7.183407, 11.064622, 5.930222, 5.473242, 2.193797]
prob = [size[0] / (size[0] + 5.660569), size[1] / (size[1] + 6.922555), size[2] / (size[2] + 6.504332),
        size[3] / (size[3] + 6.165049), size[4] / (size[4] + 5.816060), size[5] / (size[5] + 3.326408),
        size[6] / (size[6] + 3.426814)]
d = list(range(21))

In [15]:
# Value function for the corresponding non-perishable inventory problem
mdp = NonPerishableInventoryMDP()
VFDic = NonPerishableValueIteration(mdp, demand=d, size=size, prob=prob, cs=cs)
VF = pd.DataFrame.from_dict(VFDic, orient='columns')

## Deter. Policy

In [16]:
# read out the optimal policy
open_file = open("DetPolicy_m5_CS.pkl", "rb")
DetPolicy = pickle.load(open_file)
open_file.close()

In [17]:
DetPolicy[3]['CS']

'[10, 1, 20, 2]'

In [24]:
pi = DetPolicy[3]['OP']
Vhat = [{} for i in range(7)]

## LB Policy

In [19]:
# read out the optimal policy
open_file = open("InfoRelaxRandHorizonLength_m5_Rep4000_CaseStudy_Cost101202.pkl", "rb")
result = pickle.load(open_file)
open_file.close()

In [20]:
pi = [{} for i in range(7)]
Vhat = result

## Out-of-Sample Performance

In [21]:
def ExpCost(pi, demand, alpha = 0.95, replication = 100):
    result = []
    rng6 = np.random.RandomState(seed=5)
    rng7 = np.random.RandomState(seed=6)
    
    N = len(demand)
    
    U = rng7.random((replication, N, 20))
    
    f = np.zeros((N, replication))
    h = np.zeros((N, replication))
    l = np.zeros((N, replication)) 
    w = np.zeros((N, replication))
    x = np.zeros((N, replication))
    q = np.zeros((N, replication))

    for rep in tqdm(range(replication)):
                
        state = [0 for i in range(m-1)]
        cost = 0
        for t in range (N):
            day = t % 7
            # Find the order size that minimizes the expected discounted cost
            if str(str(state)) in pi[day]:
                order = pi[day][str(state)]
            else:    
                order = ApproximateDecision(day, state, b)
                pi[day][str(state)] = order
            
            x[t, rep] = order
            q[t,rep] = sum(state) + order
            # Generate initial ages
            P = []
            for i in range(m):
                if i < m-1:
                    P.append(np.exp(c0_test[i] + c1_test[i] * order)/(1 + sum(np.exp(c0_test[j] + c1_test[j] * order) for j in range(m-1))))
                else:
                    P.insert(0, 1-sum(P))         
            y = RV_Multinomial(order, P, U[rep][t])

            f[t, rep] = int(order>0)
            h[t, rep] = max(order + sum(state) - demand[t], 0)
            l[t, rep] = max(demand[t] - order - sum(state), 0)
            w[t, rep] = max(state[m-2] + y[0] - demand[t], 0)
           # Update the inventory state
            newState = [0 for i in range(m-1)]
            for j in range(m-2):
                newState[m-2-j] = max(state[m-3-j] + y[j+1] - max(demand[t] - sum(state[m-2-k] + y[k] for k in range(j+1)), 0), 0)
            newState[0] = max(y[m-1] - max(demand[t] - sum(state[m-2-k] + y[k] for k in range(m-1)), 0), 0)
            cost += pow(alpha, t)*(cs[0]*int(order>0) + cs[1]*max(order + sum(state) - demand[t], 0) + cs[2]*max(demand[t] - order - sum(state), 0) + cs[3]*max(state[m-2] + y[0] - demand[t], 0))
            state = newState
            
        result.append(cost)  
     
    return result, f, h, l, w, x, q    

In [22]:
m = 5
cs = [10, 1, 20, 2]
Plogit_test = [1.9, 3.1, 3.1, 2.5, -0.03, -0.06, -0.03, -0.09]
c0_test = [Plogit_test[0], Plogit_test[1], Plogit_test[2], Plogit_test[3]]
c1_test = [Plogit_test[4], Plogit_test[5], Plogit_test[6], Plogit_test[7]]
size = [3.497361, 10.985837, 7.183407, 11.064622, 5.930222, 5.473242, 2.193797]
prob = [size[0] / (size[0] + 5.660569), size[1] / (size[1] + 6.922555), size[2] / (size[2] + 6.504332),
        size[3] / (size[3] + 6.165049), size[4] / (size[4] + 5.816060), size[5] / (size[5] + 3.326408),
        size[6] / (size[6] + 3.426814)]
d = list(range(21))

In [23]:
# Simulate out-of-sample demand data

# Generate the sequence of dates
dates = pd.date_range(start="2017-01-01", end="2017-12-31", freq="D")

# Generate random demand for each day
np.random.seed(123)  # For reproducibility
y = [np.random.negative_binomial(size[day], prob[day]) for day in dates.weekday]

In [25]:
Cost, fixed, holding, shortage, wastage, ordersize, targetlevels = ExpCost(pi , demand = y, alpha = 0.95, replication = 100)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 31.81it/s]


In [26]:
mean_confidence_interval_95(fixed.sum(axis=0)*100/365)

(44.81643835616438, 44.744766994650234, 44.88810971767853, 0.07167136151415161)

In [27]:
mean_confidence_interval_95(shortage.sum(axis = 0) + ordersize.sum(axis = 0))

(2014.62, 2013.2971653601282, 2015.9428346398715, 1.3228346398716828)

In [28]:
mean_confidence_interval_95(holding.sum(axis=0)/365)

(7.758794520547946, 7.750816601677014, 7.766772439418878, 0.00797791887093213)

In [29]:
mean_confidence_interval_95(shortage.sum(axis = 0)/(shortage.sum(axis = 0) + ordersize.sum(axis = 0))*100)

(3.2174221973760706,
 3.169427649779203,
 3.265416744972938,
 0.047994547596867713)

In [30]:
mean_confidence_interval_95(wastage.sum(axis = 0)/(shortage.sum(axis = 0) + ordersize.sum(axis = 0))*100)

(4.4553387923575585,
 4.3906687210431885,
 4.5200088636719284,
 0.06467007131436998)

In [31]:
mean_confidence_interval_95(Cost)

(295.99441212547475, 292.30963316702395, 299.67919108392556, 3.684778958450829)

In [32]:
pd.DataFrame(targetlevels)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,...,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
1,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
2,17.0,17.0,17.0,17.0,17.0,16.0,17.0,16.0,17.0,17.0,...,17.0,16.0,16.0,17.0,16.0,16.0,17.0,16.0,17.0,17.0
3,8.0,8.0,8.0,8.0,8.0,16.0,8.0,16.0,8.0,8.0,...,8.0,16.0,16.0,8.0,16.0,16.0,8.0,16.0,8.0,8.0
4,14.0,14.0,14.0,14.0,14.0,12.0,14.0,12.0,14.0,14.0,...,14.0,12.0,12.0,14.0,12.0,12.0,14.0,12.0,14.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,...,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
361,15.0,15.0,15.0,15.0,12.0,14.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
362,11.0,12.0,12.0,12.0,9.0,11.0,12.0,12.0,12.0,12.0,...,12.0,12.0,11.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0
363,15.0,16.0,16.0,17.0,15.0,16.0,16.0,16.0,17.0,16.0,...,16.0,17.0,16.0,16.0,17.0,16.0,16.0,16.0,16.0,16.0


## $(s, S)$ Policy

In [33]:
# Function to calculate the expected cost
def ExpCost_sS(pi, alpha = 0.95, N = 365, replication = 1000):
    result = []
    rng1 = np.random.RandomState(seed=5)
    rng2 = np.random.RandomState(seed=6)
    
    U = rng2.random((replication, N, 20))
    
    f = np.zeros((N, replication))
    h = np.zeros((N, replication))
    l = np.zeros((N, replication)) 
    w = np.zeros((N, replication))
    x = np.zeros((N, replication))
    q = np.zeros((N, replication))

    for rep in range(replication):
                
        state = [0 for i in range(m-1)]
        cost = 0
        for t in range (N):
            day = t % 7

            if sum(state) < int(pi[0]):
                order = int(pi[1]) - sum(state)
            else: 
                order = 0
            
            x[t, rep] = order
            q[t,rep] = sum(state) + order
            # Generate initial ages
            P = []
            for i in range(m):
                if i < m-1:
                    P.append(np.exp(c0_test[i] + c1_test[i] * order)/(1 + sum(np.exp(c0_test[j] + c1_test[j] * order) for j in range(m-1))))
                else:
                    P.insert(0, 1-sum(P))         
            y = RV_Multinomial(order, P, U[rep][t])
            
            # Generate demand
            demand = rng1.negative_binomial(size[day], prob[day])
            if demand > 20:
                demand = 20

            f[t, rep] = int(order>0)
            h[t, rep] = max(order + sum(state) - demand, 0)
            l[t, rep] = max(demand - order - sum(state), 0)
            w[t, rep] = max(state[m-2] + y[0] - demand, 0)
           # Update the inventory state
            newState = [0 for i in range(m-1)]
            for j in range(m-2):
                newState[m-2-j] = max(state[m-3-j] + y[j+1] - max(demand - sum(state[m-2-k] + y[k] for k in range(j+1)), 0), 0)
            newState[0] = max(y[m-1] - max(demand - sum(state[m-2-k] + y[k] for k in range(m-1)), 0), 0)
            cost += pow(alpha, t)*(cs[0]*int(order>0) + cs[1]*max(order + sum(state) - demand, 0) + cs[2]*max(demand - order - sum(state), 0) + cs[3]*max(state[m-2] + y[0] - demand, 0))
            state = newState
            
        result.append(cost)  
     
    return result   

In [34]:
m = 5
cs = [10, 1, 20, 2]
c0_test = [1.9, 3.1, 3.1, 2.5]
c1_test = [-0.03, -0.06, -0.03, -0.09]
size = [3.497361, 10.985837, 7.183407, 11.064622, 5.930222, 5.473242, 2.193797]
prob = [size[0] / (size[0] + 5.660569), size[1] / (size[1] + 6.922555), size[2] / (size[2] + 6.504332),
        size[3] / (size[3] + 6.165049), size[4] / (size[4] + 5.816060), size[5] / (size[5] + 3.326408),
        size[6] / (size[6] + 3.426814)]

In [35]:
# Define grid search for (s, S) parameters
s_min = 1
s_max = 20
S_min = 1
S_max = 20
increment_size = 1

s_values = np.linspace(s_min, s_max, int((s_max-s_min)/increment_size)+1)
S_values = np.linspace(S_min, S_max, int((S_max-S_min)/increment_size)+1)

grid_search_sS_values =[]
for x in s_values:
    for y in S_values:
        sS = (x, y)
        grid_search_sS_values.append(sS)

In [36]:
grid_search_sS_values_relevant = []
for sS in grid_search_sS_values:
    if (sS[0] < sS[1]):
        grid_search_sS_values_relevant.append(sS)

In [ ]:
%%time
ECost = []
for sS in tqdm(grid_search_sS_values_relevant):
    Cost = ExpCost_sS(sS)
    ECost.append(mean_confidence_interval_95(Cost)[0])

In [ ]:
ECost.index(min(ECost))

In [ ]:
# Extract (s, S) policy minimizing the expected cost
grid_search_sS_values_relevant[ECost.index(min(ECost))]

## Out-of-Sample Performance

In [37]:
def ExpCost_sS(pi, demand, alpha = 0.95, replication = 100):
    result = []
    rng6 = np.random.RandomState(seed=5)
    rng7 = np.random.RandomState(seed=6)
    
    N = len(demand)
    
    U = rng7.random((replication, N, 20))
    
    f = np.zeros((N, replication))
    h = np.zeros((N, replication))
    l = np.zeros((N, replication)) 
    w = np.zeros((N, replication))
    x = np.zeros((N, replication))
    q = np.zeros((N, replication))

    for rep in range(replication):
                
        state = [0 for i in range(m-1)]
        cost = 0
        for t in range (N):
            day = t % 7
            
            if sum(state) < int(pi[0]):
                order = int(pi[1]) - sum(state)
            else: 
                order = 0
            
            x[t, rep] = order
            q[t,rep] = sum(state) + order
            # Generate initial ages
            P = []
            for i in range(m):
                if i < m-1:
                    P.append(np.exp(c0_test[i] + c1_test[i] * order)/(1 + sum(np.exp(c0_test[j] + c1_test[j] * order) for j in range(m-1))))
                else:
                    P.insert(0, 1-sum(P))         
            y = RV_Multinomial(order, P, U[rep][t])

            f[t, rep] = int(order>0)
            h[t, rep] = max(order + sum(state) - demand[t], 0)
            l[t, rep] = max(demand[t] - order - sum(state), 0)
            w[t, rep] = max(state[m-2] + y[0] - demand[t], 0)
           # Update the inventory state
            newState = [0 for i in range(m-1)]
            for j in range(m-2):
                newState[m-2-j] = max(state[m-3-j] + y[j+1] - max(demand[t] - sum(state[m-2-k] + y[k] for k in range(j+1)), 0), 0)
            newState[0] = max(y[m-1] - max(demand[t] - sum(state[m-2-k] + y[k] for k in range(m-1)), 0), 0)
            cost += pow(alpha, t)*(cs[0]*int(order>0) + cs[1]*max(order + sum(state) - demand[t], 0) + cs[2]*max(demand[t] - order - sum(state), 0) + cs[3]*max(state[m-2] + y[0] - demand[t], 0))
            state = newState
            
        result.append(cost)  
     
    return result, f, h, l, w, x, q    

In [38]:
m = 5
cs = [10, 1, 20, 2]
c0_test = [1.9, 3.1, 3.1, 2.5]
c1_test = [-0.03, -0.06, -0.03, -0.09]
size = [3.497361, 10.985837, 7.183407, 11.064622, 5.930222, 5.473242, 2.193797]
prob = [size[0] / (size[0] + 5.660569), size[1] / (size[1] + 6.922555), size[2] / (size[2] + 6.504332),
        size[3] / (size[3] + 6.165049), size[4] / (size[4] + 5.816060), size[5] / (size[5] + 3.326408),
        size[6] / (size[6] + 3.426814)]

In [39]:
# Simulate out-of-sample demand data

# Generate the sequence of dates
dates = pd.date_range(start="2017-01-01", end="2017-12-31", freq="D")

# Generate random demand for each day
np.random.seed(123)  # For reproducibility
y = [np.random.negative_binomial(size[day], prob[day]) for day in dates.weekday]

In [40]:
# cs = [10, 1, 20, 2]
pi = (8.0, 15.0)

In [41]:
# cs = [20, 1, 20, 20]
# pi = (6.0, 14.0)

In [42]:
# cs = [20, 1, 20, 5] / [20, 1, 20, 2]
# pi = (7.0, 18.0)

In [43]:
# cs = [10, 1, 20, 5]
# pi = (8.0, 14.0)

In [44]:
# cs = [10, 1, 20, 20]
# pi = (7.0, 12.0)

In [45]:
Cost, fixed, holding, shortage, wastage, ordersize, targetlevels = ExpCost_sS(pi , demand = y, alpha = 0.95, replication = 100)

In [46]:
mean_confidence_interval_95(fixed.sum(axis=0)*100/365)

(49.980821917808214, 49.89205783461278, 50.06958600100365, 0.08876408319543455)

In [47]:
mean_confidence_interval_95(shortage.sum(axis = 0) + ordersize.sum(axis = 0))

(2033.71, 2032.2951709121317, 2035.1248290878684, 1.4148290878684917)

In [48]:
mean_confidence_interval_95(holding.sum(axis=0)/365)

(7.669452054794521,
 7.6640269466959445,
 7.674877162893097,
 0.005425108098575839)

In [49]:
mean_confidence_interval_95(shortage.sum(axis = 0)/(shortage.sum(axis = 0) + ordersize.sum(axis = 0))*100)

(1.8149480016869797,
 1.769095396812345,
 1.8608006065616143,
 0.045852604874634675)

In [50]:
mean_confidence_interval_95(wastage.sum(axis = 0)/(shortage.sum(axis = 0) + ordersize.sum(axis = 0))*100)

(5.36832539390675, 5.302603103701932, 5.434047684111569, 0.06572229020481855)

In [51]:
mean_confidence_interval_95(Cost)

(293.5975551017731, 292.356669787616, 294.83844041593017, 1.2408853141570604)

In [52]:
pd.DataFrame(targetlevels)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
1,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
2,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
3,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
4,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
361,14.0,14.0,14.0,14.0,11.0,14.0,14.0,14.0,14.0,14.0,...,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
362,10.0,11.0,11.0,11.0,8.0,11.0,9.0,11.0,11.0,9.0,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,10.0,11.0
363,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
